In [ ]:
#@title
!pip install -qU datasets==2.12.0 qdrant-client==1.2.0 sentence-transformers==2.2.2 torch==2.0.1
!pip install langchain sentence_transformers
!pip install rapidocr-onnxruntime
# !pip install pypdf
!pip install ctransformers
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.2

In [ ]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings#to get embeddings
from langchain.vectorstores import Qdrant #vector database
from qdrant_client import QdrantClient
from langchain.llms import CTransformers#to get llm
from langchain.text_splitter import RecursiveCharacterTextSplitter#splitting text into chunks
from langchain.chains import RetrievalQA#building Retrieval chain
# from langchain.document_loaders import PyPDFLoader,  UnstructuredURLLoader #to read pdfs, urls
import torch
from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List
import pandas as pd

In [ ]:
def create_description_and_metadata(row):
    # Create description
    description = f"The product {row['product']} belongs to the category {row['category']} and subcategory {row['sub_category']}, manufactured by the brand {row['brand']}. It is priced at {row['sale_price']}, with a market price of {row['market_price']}. This product is of type {row['type']} and has a rating of {row['rating']}. Description: {row['description']}."

    # Create metadata
    metadata = {
        'name': row['product'],
        'category': row['category'],
        'sub_category': row['sub_category'],
        'brand': row['brand'],
        'sale_price': row['sale_price'],
        'market_price': row['market_price'],
        'type': row['type'],
        'rating': row['rating'],
        'decription' : row['description']
    }

    return description, metadata

# Set the file path
filepath = '/content/bigBasketProducts.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(filepath)

# Apply the create_description_and_metadata function to each row
df[['description', 'metadata']] = df.apply(create_description_and_metadata, axis=1, result_type='expand')

# Extract metadata list and save to a text file
metadata_list = df['metadata'].tolist()

with open('/content/bigBasketProducts_metadata_list.txt', 'w') as f:
    for metadata in metadata_list:
        f.write(str(metadata) + '\n')

# Keep only the 'description' column
df = df[['description']]

# Save the updated DataFrame to a new CSV file
df.to_csv('/content/dataset.csv', index=False)



In [ ]:
df.head()

,description
0,The product Garlic Oil - Vegetarian Capsule 50...
1,The product Water Bottle - Orange belongs to t...
2,"The product Brass Angle Deep - Plain, No.2 bel..."
3,The product Cereal Flip Lid Container/Storage ...
4,The product Creme Soft Soap - For Hands & Body...


In [ ]:
metadata = metadata_list
print(metadata[0])

{'name': 'Garlic Oil - Vegetarian Capsule 500 mg', 'category': 'Beauty & Hygiene', 'sub_category': 'Hair Care', 'brand': 'Sri Sri Ayurveda ', 'sale_price': 220.0, 'market_price': 220.0, 'type': 'Hair Oil & Serum', 'rating': 4.1, 'decription': 'This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/'}


In [ ]:
client = QdrantClient(":memory:")

In [ ]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
collections = []

In [ ]:
collection_name = "BigBasket-QnA"

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

collections=[]
collections=[CollectionDescription(name='BigBasket-QnA')]


In [ ]:
import pandas as pd

In [ ]:
# df = pd.read_csv('/content/dataset.csv')

In [ ]:
metadata = metadata_list

In [ ]:
metadata[0]

{'name': 'Garlic Oil - Vegetarian Capsule 500 mg',
 'category': 'Beauty & Hygiene',
 'sub_category': 'Hair Care',
 'brand': 'Sri Sri Ayurveda ',
 'sale_price': 220.0,
 'market_price': 220.0,
 'type': 'Hair Oil & Serum',
 'rating': 4.1,
 'decription': 'This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/'}

In [ ]:
df.head()

,description
0,The product Garlic Oil - Vegetarian Capsule 50...
1,The product Water Bottle - Orange belongs to t...
2,"The product Brass Angle Deep - Plain, No.2 bel..."
3,The product Cereal Flip Lid Container/Storage ...
4,The product Creme Soft Soap - For Hands & Body...


In [ ]:
%%time

batch_size = 512  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

for index in tqdm(range(0, len(df), batch_size)):
    i_end = min(index + batch_size, len(df))  # find end of batch
    batch = df.iloc[index:i_end]  # extract batch
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})
    emb = embeddings.embed_documents(batch["description"]) # generate embeddings for batch
    meta = metadata[index:i_end]

    ids = list(range(index, i_end))  # create unique IDs

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=emb, payloads=meta),
    )

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(df)

  0%|          | 0/54 [00:00<?, ?it/s]

Vector count in collection: 27555
CPU times: user 32min 3s, sys: 6min 31s, total: 38min 34s
Wall time: 38min 38s


In [ ]:
# snapshot_info = client.create_snapshot(collection_name=collection_name)
# print(snapshot_info)

In [ ]:
# client.upload_collection(
#     collection_name=collection_name,
#     vectors=emb,
#     batch_size=64,
#     parallel=2,
# )

In [ ]:
sentencetf = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def getInfo(question: str, top_k: int):
    """
    Get the relevant plot for a given question

    Args:
        question (str): What do we want to know?
        top_k (int): Top K results to return

    Returns:
        context (List[str]):
    """
    try:
        encoded_query =  sentencetf.encode(question)  # generate embeddings for the question

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer
        # print(result)

        context = [
    {
        'name': x.payload.get('name', ''),
        'category': x.payload.get('category', ''),
        'sub_category': x.payload.get('sub_category', ''),
        'brand': x.payload.get('brand', ''),
        'sale_price': x.payload.get('sale_price', 0.0),
        'market_price': x.payload.get('market_price', 0.0),
        'type': x.payload.get('type', ''),
        'rating': x.payload.get('rating', float('nan'))
        # 'description': x.payload.get('description', '')
    } for x in result
]
        input_data = {
            'context': context,
            'question': question,


        }
        prompt = PromptTemplate.from_template(
        "question is={question}. you should answer from: {context}"
        # print(context)
)


        # Chain
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        answer = llm_chain(input_data)
        return answer

    except Exception as e:
        print({e})

In [ ]:
getInfo('what is the price of chia seeds?', 1)

{'context': [{'name': 'Chia Seeds',
   'category': 'Gourmet & World Food',
   'sub_category': 'Snacks, Dry Fruits, Nuts',
   'brand': 'NUTRASHIL',
   'sale_price': 145.25,
   'market_price': 175.0,
   'type': 'Dry Fruits & Berries',
   'rating': nan}],
 'question': 'what is the price of chia seeds?',
 'text': '\n\nAnswer: The price of chia seeds is 145.25 Rupees.'}

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
def load_llm():
    llm = CTransformers(
        model = "TheBloke/Llama-2-7B-Chat-GGML",
        model_type="llama",
        temperature = 0.2
        )
    return llm

In [ ]:
llm = load_llm()
# llm_chain = LLMChain(llm=llm, prompt=prompt)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
custom_prompt_template = """Use the following pieces of information to answer the user’s question.
If you don’t know the answer, just say that you don’t know, don’t try to make up an answer.

Context: {context}
Question: {question}

"""

prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])

In [ ]:
def qa_bot_qdrant_response(context):
    # embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
    #                                    model_kwargs={'device': 'cpu'})
    #connect to the vector database
    # client = QdrantClient(url=qdrant_url,api_key=qdrant_api_key)

    doc_store = Qdrant(
        client=client,
        collection_name="BigBasket-QnA",
        embeddings=embeddings)

    llm = load_llm()
    qa = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=doc_store.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=False,
                                       chain_type_kwargs={'prompt': prompt}
                                       )

    response = qa({'query': context})

    return response

In [ ]:
# response('')

In [ ]:
from langchain.llms import GPT4All

In [ ]:
class ConversationManager:
    def __init__(self):
        self.context = ""  # Initialize empty context for the conversation

    def update_context(self, new_context):
        self.context += " " + new_context  # Append new context to the existing context

    def get_response(self, query):
        self.update_context(query)  # Add the user's query to the conversation context
        response = qa_bot_qdrant_response(query)
        return response


In [ ]:
conversation_manager = ConversationManager()

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        print("Bot: Goodbye!")
        break
    response = conversation_manager.get_response(user_input)
    print("Bot:", response)